## Unet Model - 2015 NY datasets Deep <br>

### Date : 2015-01-01 ~ 2015-03-01 
### training : 40 day (20% val ) / 20 day 
#### Summary : 

----------------------



In [1]:

import pandas as pd
import numpy as np
import tensorflow as tf

import random as rn
from utils import *
import os
os.environ['PYTHONHASHSEED'] = '0'

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
rn.seed(RANDOM_SEED)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

import keras
import keras.backend as K

tf.set_random_seed(RANDOM_SEED)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

from keras import layers
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, AveragePooling2D, UpSampling2D, Flatten, Reshape, Dropout, Conv2DTranspose
from keras.layers import Concatenate, BatchNormalization, Add
from keras.models import Model, Sequential
from keras.layers import InputLayer
from keras.utils.training_utils import multi_gpu_model


###################
## Hyperparam  - 맞게 수정하거나 Arg로 받아 온다. (Data Folder / Model Name / Model Ver ) 추가 
###################


LRATE = 1e-2
BATCH_SIZE = 128
EPOCHS = 5
SCALE = 'log' # or 'max' or 'log'
NGPU = 2

## Folder / File Name
DATA_FOLDER = '/data/public/rw/prj-mobility/datasets/NY_data/2ch/'

## Baseline Model Folder 
BASE_MODEL_NAME = 'MODEL_V1_NYC'
MODEL_VER = '0827_NY_001_2ch_test_base_1'


##################################################
OUTPUT_FOLDER = '../output_file/'
TF_FOLDER = '../tfgraph/'
MODEL_SAVE_FOLDER = '../saved_model/'

MODEL_NAME_ = BASE_MODEL_NAME+'_'+MODEL_VER
OUTPUT_FOLDER = OUTPUT_FOLDER+BASE_MODEL_NAME+'/'
TF_FOLDER = TF_FOLDER+BASE_MODEL_NAME+'/'+MODEL_VER
MODEL_SAVE_FOLDER = MODEL_SAVE_FOLDER+BASE_MODEL_NAME+'/'

print ('Model Name :',MODEL_NAME_)

if os.path.isdir(OUTPUT_FOLDER) == False:
    os.makedirs(OUTPUT_FOLDER)
    print ('Output Folder :',OUTPUT_FOLDER, 'created')
else :
    print ('Output Folder :',OUTPUT_FOLDER)

if os.path.isdir(TF_FOLDER) == False:
    os.makedirs(TF_FOLDER)
    print ('Tfgraph Folder :',TF_FOLDER, 'created')
else :
    print ('Tfgraph Folder :',TF_FOLDER)
    
if os.path.isdir(MODEL_SAVE_FOLDER) == False:
    os.makedirs(MODEL_SAVE_FOLDER)
    print ('Save Folder :',MODEL_SAVE_FOLDER, 'created')
else :
    print ('Save Folder :',MODEL_SAVE_FOLDER)




Using TensorFlow backend.


Model Name : MODEL_V1_NYC_0827_NY_001_2ch_test_base_1
Output Folder : ../output_file/MODEL_V1_NYC/
Tfgraph Folder : ../tfgraph/MODEL_V1_NYC/0827_NY_001_2ch_test_base_1
Save Folder : ../saved_model/MODEL_V1_NYC/


In [2]:
########################################
## Data Load
########################################


NYC_FOLDER = '/data/public/rw/prj-mobility/10.195.12.143/kakaobrain/taxi/datasets/NY_data/'

x_st_train = np.log(1.0+load_np_data(NYC_FOLDER+'train_st_x_2d.npz'))
x_ed_train = np.log(1.0+load_np_data(NYC_FOLDER+'train_end_x_2d.npz'))
y_train = np.log(1.0+load_np_data(NYC_FOLDER+'train_st_y_2d.npz'))
temporal_train = load_np_data(NYC_FOLDER+'temporal_train.npz')


x_st_test = np.log(1.0+load_np_data(NYC_FOLDER+'test_st_x_2d.npz'))
x_ed_test = np.log(1.0+load_np_data(NYC_FOLDER+'test_end_x_2d.npz'))
y_test = np.log(1.0+load_np_data(NYC_FOLDER+'test_st_y_2d.npz'))
temporal_test = load_np_data(NYC_FOLDER+'temporal_test.npz')

coord_train = load_np_data(NYC_FOLDER+'train_coord_x_2d.npz')/10 #args.coord
coord_test = load_np_data(NYC_FOLDER+'test_coord_x_2d.npz')/10 #args.coord

x_st_train = x_st_train[:-1]
x_ed_train = x_ed_train[:-1]

x_st_test = x_st_test[:-1]
x_ed_test = x_ed_test[:-1]

y_t2_train = y_train[1:]
y_t2_test = y_test[1:]

y_train = y_train[:-1]
y_test = y_test[:-1]

x_t2_temporal_train = temporal_train[1:]
x_t2_temporal_test = temporal_test[1:]

x_t1_temporal_train = temporal_train[:-1]
x_t1_temporal_test = temporal_test[:-1]

x_st_train = np.concatenate((x_st_train,coord_train[:-1]), axis=3)
x_st_test = np.concatenate((x_st_test,coord_test[:-1]), axis=3)

#y_train = np.concatenate( (y_train, y_t2_train), axis=3)
#y_test = np.concatenate( (y_test, y_t2_test), axis=3)

print (x_st_train.shape, x_ed_train.shape, y_train.shape, x_t1_temporal_train.shape,x_t2_temporal_train.shape, y_t2_train.shape)
print (x_st_test.shape, x_ed_test.shape, y_test.shape, x_t1_temporal_test.shape,x_t2_temporal_test.shape, y_t2_test.shape)


/data/public/rw/prj-mobility/10.195.12.143/kakaobrain/taxi/datasets/NY_data/train_st_x_2d.npz Loaded
/data/public/rw/prj-mobility/10.195.12.143/kakaobrain/taxi/datasets/NY_data/train_end_x_2d.npz Loaded
/data/public/rw/prj-mobility/10.195.12.143/kakaobrain/taxi/datasets/NY_data/train_st_y_2d.npz Loaded
/data/public/rw/prj-mobility/10.195.12.143/kakaobrain/taxi/datasets/NY_data/temporal_train.npz Loaded
/data/public/rw/prj-mobility/10.195.12.143/kakaobrain/taxi/datasets/NY_data/test_st_x_2d.npz Loaded
/data/public/rw/prj-mobility/10.195.12.143/kakaobrain/taxi/datasets/NY_data/test_end_x_2d.npz Loaded
/data/public/rw/prj-mobility/10.195.12.143/kakaobrain/taxi/datasets/NY_data/test_st_y_2d.npz Loaded
/data/public/rw/prj-mobility/10.195.12.143/kakaobrain/taxi/datasets/NY_data/temporal_test.npz Loaded
/data/public/rw/prj-mobility/10.195.12.143/kakaobrain/taxi/datasets/NY_data/train_coord_x_2d.npz Loaded
/data/public/rw/prj-mobility/10.195.12.143/kakaobrain/taxi/datasets/NY_data/test_coord_x

## Data verification

In [3]:
print (np.average(np.equal(x_st_train[1,:,:,-3], y_train[0,:,:,0])))
print (np.average(np.equal(x_st_train[0,:,:,-3], x_st_train[1,:,:,-4])))

1.0
1.0


In [4]:
## Make Validation Data About 20% 
val_idx = 1524

print ('----- Before Split ----')
#print (x_st_train.shape, x_ed_train.shape, x_t1_temporal_train.shape, x_t0_temporal_train.shape, x_t2_temporal_train.shape)

x_st_val = x_st_train[val_idx:]
x_ed_val = x_ed_train[val_idx:]
y_val = y_train[val_idx:]
x_t1_temporal_val = x_t1_temporal_train[val_idx:]
#x_t0_temporal_val = x_t0_temporal_train[val_idx:]
x_t2_temporal_val = x_t2_temporal_train[val_idx:]


x_st_train = x_st_train[:val_idx]
x_ed_train = x_ed_train[:val_idx]
y_train = y_train[:val_idx]
x_t1_temporal_train = x_t1_temporal_train[:val_idx]
#x_t0_temporal_train = x_t0_temporal_train[:val_idx]
x_t2_temporal_train = x_t2_temporal_train[:val_idx]


print ('----- After Split ----')
print (x_st_train.shape, x_ed_train.shape, x_t1_temporal_train.shape, x_t2_temporal_train.shape)
print (x_st_val.shape, x_ed_val.shape, x_t1_temporal_val.shape, x_t2_temporal_val.shape)


----- Before Split ----
----- After Split ----
(1524, 10, 20, 10) (1524, 10, 20, 16) (1524, 10, 20, 57) (1524, 10, 20, 57)
(379, 10, 20, 10) (379, 10, 20, 16) (379, 10, 20, 57) (379, 10, 20, 57)


In [5]:
K.clear_session()
print ('Session Reseted')

model_input_train_data = [x_st_train, x_ed_train, x_t1_temporal_train, x_t2_temporal_train]
model_input_val_data=[x_st_val, x_ed_val, x_t1_temporal_val, x_t2_temporal_val]
model_input_test_data = [x_st_test, x_ed_test, x_t1_temporal_test, x_t2_temporal_test]


def make_temporal_model(st_data, ed_data, tmp_data):

    im_h,im_w,im_c = st_data[0].shape

    start_input = Input(shape=(st_data[0].shape))
    end_input = Input(shape=(ed_data[0].shape))
    t1_temporal_input = Input(shape=(tmp_data[0].shape))
    t2_temporal_input = Input(shape=(tmp_data[0].shape))

    input_tensors = [start_input, end_input, t1_temporal_input, t2_temporal_input]

    coord_input = keras.layers.Lambda(lambda xin: xin[:,:,:,-2:] )(start_input)
    start_input = keras.layers.Lambda(lambda xin: xin[:,:,:,:-2] )(start_input)

    #Guassian Noise Augmentation for Training

    ## 2a or 2b 추가 
    #temp_conv2d_1st = Conv2D(16, kernel_size=(1,1), strides=(1,1))
    #temp_conv2d_2nd = Conv2D(16, kernel_size=(1,1), strides=(1,1))

    #net_t1_temp = temp_conv2d_1st(t1_temporal_input)
    #net_t1_temp = temp_conv2d_2nd(net_t1_temp)
    
    #net_t2_temp = temp_conv2d_1st(t2_temporal_input)
    #net_t2_temp = temp_conv2d_2nd(net_t2_temp)

    # 1a or 2a 버전

    net_t1_temp = Conv2D(16, kernel_size=(1,1), strides=(1,1))(t1_temporal_input)
    net_t1_temp = Conv2D(16, kernel_size=(1,1), strides=(1,1))(net_t1_temp)

    #net_t2_temp = Conv2D(16, kernel_size=(1,1), strides=(1,1))(t2_temporal_input)
    #net_t2_temp = Conv2D(16, kernel_size=(1,1), strides=(1,1))(net_t2_temp)

    net1 = layers.concatenate([start_input, net_t1_temp], axis=-1)
    net1 = Conv2D(64, kernel_size=(3,3), activation=None, padding='same')(net1)
    net1 = layers.Activation('relu')(net1)
    net1 = BatchNormalization()(net1)

    net11 = AveragePooling2D(pool_size=(2,2), strides=(2,2))(net1)

    net2 = Conv2D(128, kernel_size=(3,3), activation=None, padding='same')(net11)
    net2 = layers.Activation('relu')(net2)
    net2 = BatchNormalization()(net2)

    net3 = Conv2D(128, kernel_size=(3,3), activation=None, padding='same')(net2)
    net3 = layers.Activation('relu')(net3)
    net3 = BatchNormalization()(net3)
    net33 = layers.concatenate([net2, net3], axis=-1)

    net4 = Conv2D(128, kernel_size=(3,3), activation=None, padding='same')(net33)
    net4 = layers.Activation('relu')(net4)
    net4 = BatchNormalization()(net4)
    net44 = layers.concatenate([net2, net3, net4], axis=-1)
    #net4 = layers.Add()([net2, net4])

    net5 = Conv2DTranspose(128, kernel_size=(2,2), strides=(2,2), padding='same')(net44)
    net5 = layers.Activation('relu')(net5)
    net5 = BatchNormalization()(net5)
    net5 = Dropout(0.5)(net5)

    net5 = layers.concatenate([net5, net1], axis=-1)

    net6 = Conv2DTranspose(256, kernel_size=(3,3), strides=(1,1), padding='same')(net5)
    net6 = layers.Activation('relu')(net6)
    net6 = BatchNormalization()(net6)
    net6 = Dropout(0.5)(net6)

    net_end = Conv2D(8, kernel_size=(3,3), padding='same')(end_input)
    net_end = BatchNormalization()(net_end)
    net_end = Conv2D(8, kernel_size=(3,3), padding='same')(net_end)
    net_end = BatchNormalization()(net_end)
    
    

    net71 = layers.concatenate([net6, net_t1_temp, net_end, start_input, end_input, coord_input], axis=-1)
    net71 = Conv2DTranspose(256, kernel_size=(1,1), padding='same')(net71)
    net71 = layers.Activation('relu')(net71)
    net71 = BatchNormalization()(net71)

    #net72 = layers.concatenate([net6, start_input, net_end, end_input, net_t2_temp, coord_input], axis=-1)
    #net72 = Conv2DTranspose(256, kernel_size=(1,1), padding='same')(net72)
    #net72 = layers.Activation('relu')(net72)
    #net72 = BatchNormalization()(net72)

    t1_output = Conv2D(1, kernel_size=(1,1), padding='same')(net71)
    output = layers.Activation('relu')(t1_output)

    #t2_output = Conv2D(1, kernel_size=(1,1), padding='same')(net72)
    #t2_output = layers.Activation('relu')(t2_output)

    #output = layers.concatenate([t1_output, t2_output], axis=-1)

    model = Model(inputs=input_tensors, outputs=output)
    return model



b_model = make_temporal_model(x_st_train,x_ed_train, x_t1_temporal_train)
model = multi_gpu_model(b_model, gpus=NGPU)
print (MODEL_NAME_, 'Model Created')
print (b_model.summary())


Session Reseted
MODEL_V1_NYC_0827_NY_001_2ch_test_base_1 Model Created
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 10, 20, 57)   0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 10, 20, 10)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 10, 20, 16)   928         input_3[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 10, 20, 8)    0           input_1[0][0]                    
______________________________________

In [6]:
########################################
## Callback 
########################################


from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD, Adam

from keras.callbacks import Callback

class SGDLearningRateTracker(Callback):
    def on_epoch_end(self, epoch, logs={}):
        optimizer = self.model.optimizer
        lr = K.eval(optimizer.lr) # K.eval(optimizer.lr * (1. / (1. + optimizer.decay * optimizer.iterations)))
        print('LR: {:.6f}'.format(lr))

tb_hist = keras.callbacks.TensorBoard(log_dir=TF_FOLDER, histogram_freq=0, write_graph=True, write_images=True)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_invlog_mape_tr10', min_delta=0, patience=10, verbose=0, mode='min')



## Training 

#### MSE pretraining -> MAE fine-tuning 


In [7]:

import time
start_time = time.time() 


model.compile(loss=['mean_squared_error'], optimizer=Adam(lr=LRATE,  decay=0.01), metrics=['mean_absolute_error', invlog_mape_tr10])
history = model.fit(model_input_train_data , y_train,
                  batch_size=BATCH_SIZE,
                  epochs=EPOCHS,
                  callbacks=[tb_hist, SGDLearningRateTracker()],
                  shuffle = True,
                  verbose=2
                  ,validation_data=(model_input_val_data, y_val)
                 )
    
val_loss = history.history['val_loss']
val_metric = history.history['val_mean_absolute_error']

save_model_name = MODEL_SAVE_FOLDER+MODEL_NAME_+'st1.h5'
b_model.save(save_model_name)
print ('Model Saved', save_model_name)
history_save(val_loss, val_metric, model_name =  MODEL_NAME_+'st1', output_folder=OUTPUT_FOLDER)


model.compile(loss=['mean_absolute_error'], optimizer=Adam(lr=LRATE,  decay=0.01), metrics=['mean_absolute_error', invlog_mape_tr10])
history = model.fit(model_input_train_data , y_train,
                  batch_size=BATCH_SIZE,
                  epochs=EPOCHS,
                  callbacks=[tb_hist, early_stopping, SGDLearningRateTracker()],
                  shuffle = True,
                  verbose=2
                  ,validation_data=(model_input_val_data, y_val)
                 )

val_loss2 = history.history['val_loss']
val_metric2 = history.history['val_mean_absolute_error']

end_time = time.time()
n_epochs = len(val_metric)+len(val_metric2)

print ('')
print("--- Train Time : %0.2f hour  ---" %(  (end_time - start_time)/3600  ))
print("--- # of Epochs: %0.f  ---" %( n_epochs ) )


save_model_name = MODEL_SAVE_FOLDER+MODEL_NAME_+'st2.h5'
b_model.save(save_model_name)
print ('Model Saved', save_model_name)
history_save(val_loss, val_metric, model_name =  MODEL_NAME_+'st2', output_folder=OUTPUT_FOLDER)


Train on 1524 samples, validate on 379 samples
Epoch 1/5
 - 7s - loss: 5.8302 - mean_absolute_error: 1.1699 - invlog_mape_tr10: 2331274806.2314 - val_loss: 119.3727 - val_mean_absolute_error: 6.5410 - val_invlog_mape_tr10: 23354522805743323709440.0000
LR: 0.010000
Epoch 2/5
 - 1s - loss: 0.5090 - mean_absolute_error: 0.4208 - invlog_mape_tr10: 0.7774 - val_loss: 34.8769 - val_mean_absolute_error: 3.3870 - val_invlog_mape_tr10: 893218672.0422
LR: 0.010000
Epoch 3/5
 - 1s - loss: 0.3147 - mean_absolute_error: 0.3216 - invlog_mape_tr10: 0.5877 - val_loss: 4.0851 - val_mean_absolute_error: 1.1867 - val_invlog_mape_tr10: 119.5614
LR: 0.010000
Epoch 4/5
 - 1s - loss: 0.2344 - mean_absolute_error: 0.2755 - invlog_mape_tr10: 0.4487 - val_loss: 1.1193 - val_mean_absolute_error: 0.6414 - val_invlog_mape_tr10: 6.5599
LR: 0.010000
Epoch 5/5
 - 1s - loss: 0.1907 - mean_absolute_error: 0.2476 - invlog_mape_tr10: 0.3945 - val_loss: 0.4303 - val_mean_absolute_error: 0.4175 - val_invlog_mape_tr10: 1.58

## Model Output Print

In [8]:
pred_test = model.predict(model_input_test_data)

print ('')
print ("## Test datasets Performance")
print ("- MAPE(11 or more) : %.3f"%mape_trs(inverse_logscale(y_test), inverse_logscale(pred_test), 11))
print ("- RMSE(11 or more) : %.3f"%rmse_trs(inverse_logscale(y_test), inverse_logscale(pred_test), 11))
print ('')

#print ("- T1 MAPE(11 or more) : %.3f"%mape_trs(inverse_logscale(y_test[:,:,:,:1]), inverse_logscale(pred_test[:,:,:,:1]), 11))
#print ("- T2 MAPE(11 or more) : %.3f"%mape_trs(inverse_logscale(y_test[:,:,:,-1:]), inverse_logscale(pred_test[:,:,:,-1:]), 11))




## Test datasets Performance
- MAPE(11 or more) : 0.401
- RMSE(11 or more) : 102.671



In [9]:

history_save(val_loss, val_metric, model_name =  MODEL_NAME_, output_folder=OUTPUT_FOLDER)
make_test_ouput(model, model_input_test_data , y_test, model_name=MODEL_NAME_, norm=SCALE, output_folder=OUTPUT_FOLDER)
#make_test_2ch_ouput(model, model_input_test_data , y_test, model_name=MODEL_NAME_, norm=SCALE, output_folder=OUTPUT_FOLDER)



../output_file/MODEL_V1_NYC/MODEL_V1_NYC_0827_NY_001_2ch_test_base_1_loss_metric_invmape_0.418.csv  Val loss Saved
Test True saved :  ../output_file/MODEL_V1_NYC/MODEL_V1_NYC_0827_NY_001_2ch_test_base_1_gt.csv
Test Pred saved :  ../output_file/MODEL_V1_NYC/MODEL_V1_NYC_0827_NY_001_2ch_test_base_1_pred.csv

## Model Name :  ../output_file/MODEL_V1_NYC/MODEL_V1_NYC_0827_NY_001_2ch_test_base_1

## ---- Base Metric -----
- True Max 1099 , Pred Max 1250
- True Avg 40.84 , Pred Avg 55.61
- MAPE(+1) : 0.271
- RMSE : 52.141

## ---- Trs MAPE Metric -----
- 1 or More  MAPE : 0.629
- 2 or More  MAPE : 0.533
- 3 or More  MAPE : 0.490
- 4 or More  MAPE : 0.439
- 5 or More  MAPE : 0.439
- 6 or More  MAPE : 0.418
- 7 or More  MAPE : 0.409
- 8 or More  MAPE : 0.409
- 9 or More  MAPE : 0.406
- 10 or More  MAPE : 0.403
- 11 or More  MAPE : 0.401
- 12 or More  MAPE : 0.401
- 13 or More  MAPE : 0.398
- 14 or More  MAPE : 0.398
- 15 or More  MAPE : 0.396

## ---- Trs MAA Metric -----
- Range 0 %  MAA : 52

In [11]:

save_model_name = MODEL_NAME_+MODEL_NAME_+'.h5'
b_model.save(save_model_name)
print ('Model Saved', save_model_name)


Model Saved MODEL_V1_NYC_0827_NY_001_2ch_test_base_1MODEL_V1_NYC_0827_NY_001_2ch_test_base_1.h5
